<a href="https://colab.research.google.com/github/liofval/GCI/blob/main/GCI%E3%83%98%E3%82%99%E3%83%BC%E3%82%B7%E3%83%83%E3%82%AF_%E6%BC%94%E7%BF%92_%E5%9B%9E%E5%B8%B0%E3%82%B3%E3%83%BC%E3%82%B9_%E3%83%AC%E3%83%98%E3%82%99%E3%83%AB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**回帰コース**


---


回帰コースでは、**数値を予測する機械学習モデル「線形回帰」**を取り扱い、**「データの理解→データの前処理→モデルの構築・評価」**という一連の流れを通して、各レベルに応じてデータ分析に必要な知識の習得を目指します。

#レベル4（★★★★）





---
レベル4では、標準化したデータでモデルを学習し、予測をします。

##4.1.データの前処理

これまで通り、以下のリンクから自動車価格のデータをダウンロードし、所定の位置にアップロードし、ファイルをデータフレームに読み込んで欠損値を削除しましょう。

[自動車価格Data.csv](https://drive.google.com/file/d/10gmRMbUVuDF8osJGdIop2b58PWDh7ezp/view?usp=drive_link)

In [ ]:
import pandas as pd

df = pd.read_csv('自動車価格Data.csv')
df = df.dropna()

例えば、説明変数間で単位や大きさが異なる場合、各変数のうちどの変数がモデルの予測に有効であったか評価できなくなります。例えば、線形回帰では回帰係数の大小が意味を持たなくなってしまいます。kg表記のものがg表記になるだけで、回帰係数も変わってしまいます。

そこで、**標準化**と呼ばれる数値のスケーリングを行うことで、変数間の大小関係を無くし、モデルの解釈性を向上させます。

標準化を実施するには、各データに対して、その変数列の平均を引き、標準偏差で割ります。そうすることで、各説明変数が平均０、標準偏差１となるような分布を取ることになります。

具体的には、以下のような式で計算されます。ただし $\mu_x$ と $\sigma_x$ はそれぞれ 各データ$x$ の平均値と標準偏差です。
\begin{equation}
x_{std} = \cfrac{x - \mu_x}{\sigma_x}
\end{equation}

データを標準化するには`sklearn.preprocessing`の`StandardScaler`クラスを使用します。実際に先ほどの自動車価格データを使用して、標準化の方法について見てみましょう。

説明変数である`engine-size` 、`curb-weight`、`city-mpg`の平均値と標準偏差を見ると、スケールがバラバラになっていることがわかります。このままだとモデルの解釈ができません。

In [ ]:
df[['engine-size', 'curb-weight', 'city-mpg']].describe().loc[['mean', 'std']]

,engine-size,curb-weight,city-mpg
mean,126.875622,2555.666667,25.179104
std,41.546834,517.296727,6.423220


そこで、以下のように標準化を施します。`fit`メソッドは、データの平均値と標準偏差を計算します。`transform`メソッドは、すでに計算された平均値と標準偏差を使用して、データを標準化します。

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df[['engine-size', 'curb-weight', 'city-mpg']]

ss = StandardScaler()
ss.fit(X)
X_std = ss.transform(X)

# fitとtransformは以下のようにまとめられる
# X_std = ss.fit_transform(X)

df[['engine-size std', 'curb-weight std', 'city-mpg std']] = X_std

すると以下のように標準化によって、各説明変数の平均値が０、標準偏差が１となり、スケールが揃いました。今回は外れ値は除去していませんが、標準化も外れ値の影響を受けるので、本来は外れ値除去を行なってから標準化を行った方が良いです。

In [ ]:
df[['engine-size std', 'curb-weight std', 'city-mpg std']].describe().loc[['mean', 'std']]

,engine-size std,curb-weight std,city-mpg std
mean,-3.755978e-17,3.004783e-16,5.302558e-17
std,1.002497e+00,1.002497e+00,1.002497e+00


続いて線形回帰モデルを構築して、その回帰係数を見ることで、標準化によって、モデルの予測に寄与している説明変数が変化することを見てみましょう。ここで重要な点としては、訓練データに対して`fit`メソッドで平均値や標準化を算出し、それを`transform`メソッドによって訓練データとテストデータに対して標準化を行うということです。これは「データ漏洩」の防止につながります。テストデータの平均値や標準化を使ってスケーリングを行うと、モデルが本来知り得ない情報を使ってしまうことになります。これを「データ漏洩」と呼び、モデルの評価が正しく行えなくなります。

訓練データとテストデータに分けるために`scikit-learn`の`train_test_split`関数の使用を使用しています。説明変数が含まれたデータXを訓練データX_trainとテストデータX_testに、目的変数が含まれたデータyを訓練データy_trainとテストデータy_testに分割しています。デフォルトでは、訓練データ:テストデータ=3:1に分割しますが、今回はtest_size=0.2として、訓練データ:テストデータ=4:1としています。`train_test_split`関数の引数`test_size`によって、訓練データとテストデータの分割の割合を変更できます。

In [ ]:
from sklearn.model_selection import train_test_split

X = df[['engine-size', 'curb-weight', 'city-mpg']]
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ss = StandardScaler()
ss.fit(X_train)
X_train_std = ss.transform(X_train)
X_test_std = ss.transform(X_test)

##4.2.モデルの構築

まずは、標準化前のデータに対してモデルを構築します。いつもと同様に、`fit`メソッドを使用してモデルの学習を行います。

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)
print('回帰係数:', model.coef_)

回帰係数: [  96.49537458    3.56641101 -152.03978154]


回帰係数についても出力すると、3つ目の説明変数である`city-mpg`がモデルの予測に最も寄与していることになります。しかし、実際には各変数のスケールが揃っていないので、これは正しくありません。

続いて標準化後のデータに対してモデルを構築します。

In [ ]:
model = LinearRegression()
model.fit(X_train_std, y_train)
print('回帰係数:', model.coef_)

回帰係数: [3824.32832662 1748.78601005 -958.81499881]


同様に回帰係数を出力すると、１つ目の説明変数である`engine-size`がモデルの予測に最も寄与していることがわかりました。スケールを揃えることで、回帰係数の比較を行うことが可能になり、モデルの解釈ができるようになりました。

##4.3.演習問題

問4-1.レベル1で扱った「回帰コースLv1演習Data.csv」をダウンロードしてファイルをアップロードし、pandasのデータフレームとして読み込んでください。
[回帰コースLv1演習Data.csv](https://drive.google.com/file/d/1uW2cKGLxyduR0XcU_zRguHBHf35nz4gS/view?usp=drive_link)

なお、このデータは欠損値が含まれるので、欠損値が含まれる行をまず削除してください。続いてこのデータの変数である`engine-size`に対して標準化を行なってください。

問4-2.`price`を目的変数として線形回帰を行い、回帰係数を出力してください。